In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = train = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/train.csv', low_memory= False)
train.head()

In [ ]:
target = train.iloc[:,-1:]
train=train.drop('target',1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train,X_valid, y_train, y_valid = train_test_split(train,target)

In [ ]:
test= pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/test.csv', low_memory= False)
test.head()

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test= scaler.transform(test)

In [ ]:
import tensorflow as tf
from tensorflow import keras
model = keras.models.Sequential([keras.layers.Dense(30,activation="relu", input_shape= X_train.shape[1:]),
                               keras.layers.Dense(1, activation="sigmoid")])
model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=[tf.keras.metrics.AUC()])
history= model.fit(X_train,y_train, epochs=20, validation_data = (X_valid,y_valid))



In [ ]:
model.predict(X_test)

In [ ]:
#import tensorflow as tf
#from tensorflow import keras
#from scipy.stats import reciprocal
#from sklearn.model_selection import  RandomizedSearchCV

#Fine tuning Neural Network Hyperparameters using RandommizedSearch CV -- took too long to provide a result- not using it
#def build_model(n_hidden=1, n_neurons =30, learning_rate = 3e-3, input_shape=X_train.shape[1:]):
   # model = keras.models.Sequential()
   # model.add(keras.layers.InputLayer(input_shape=input_shape))
   # for layer in range(n_hidden):
       # model.add(keras.layers.Dense(n_neurons, activation="relu"))
    #model.add(keras.layers.Dense(1, activation = "sigmoid"))
    #optimizer= keras.optimizers.SGD(learning_rate=learning_rate)
   # model.compile(loss="binary_crossentropy", optimizer = optimizer , metrics =[tf.keras.metrics.AUC()])
    #return model
#keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

#param_distribs ={
   # "n_hidden" :[1,2,3],
   # "n_neurons" : np.arange(1,100),
   # "learning_rate":reciprocal(3e-4,3e-2)
}

#rnd_search_cv = RandomizedSearchCV(keras_reg,param_distribs,n_iter=1,cv=2,n_jobs=-1)
#rnd_search_cv.fit(X_train,y_train,epochs=40, validation_data = (X_valid,y_valid),
                 #callbacks=[keras.callbacks.EarlyStopping(patience=10)])


In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

#Fine tuning Neural Network Hyperparameters using keras tuners
def build_model(hp):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=X_train.shape[1:]))
   
    hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
    hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units1, activation='relu'))
    model.add(tf.keras.layers.Dense(units=hp_units2, activation='relu'))
    model.add(keras.layers.Dense(1, activation = "sigmoid"))
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    optimizer= keras.optimizers.SGD(learning_rate=hp_learning_rate)
    model.compile(loss="binary_crossentropy", optimizer = optimizer , metrics =[tf.keras.metrics.AUC()])
    return model
tuner = kt.Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo'
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(X_train, y_train, epochs=10, validation_data = (X_valid,y_valid))




In [ ]:
best_hp=tuner.get_best_hyperparameters()[0]
h_model = tuner.hypermodel.build(best_hp)
h_model.summary()
h_model.fit(X_train, y_train, epochs=20,validation_data = (X_valid,y_valid) )

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv', low_memory= False)
sample_submission.head()

In [ ]:
sample_submission[sample_submission.columns[1]] =h_model.predict(X_test)

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.to_csv('/kaggle/working/submission_nn2.csv', index=False)